<h1><center>IMT2118 - Ciencia de Datos Geoespaciales (2024-1) </center></h1>
<h2><center>Proyecto:   </center></h2>

### Imports

In [8]:
import pandas as pd
import numpy as np 
import geopandas as gpd
import matplotlib.pyplot as plt
import geemap, ee
from shapely.geometry import LineString

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize()

### Lectura de Datos

In [9]:
stgo = gpd.read_file("Datos/santiago_urbano.gpkg")
roi_shapely = stgo.union_all()

In [10]:
Areas_Verdes = gpd.read_file("Datos/Areas_Verdes.gpkg")
Bomberos = gpd.read_file("Datos/Bomberos.gpkg")
Censo_2017 = gpd.read_file("Datos/Censo_2017.gpkg")
Comercio = gpd.read_file("Datos/Comercio.gpkg")
Ejes = gpd.read_file("Datos/Ejes.gpkg")
EstablecimientosSalud = gpd.read_file("Datos/EstablecimientosSalud.gpkg")
Limite_Comunal = gpd.read_file("Datos/Limite_Comunal.gpkg")
Manzanas = gpd.read_file("Datos/Manzanas.gpkg")
Paraderos = gpd.read_file("Datos/Paraderos.gpkg")

In [11]:
Areas_Verdes = Areas_Verdes[Areas_Verdes.geometry.within(roi_shapely)]
Bomberos = Bomberos[Bomberos.geometry.within(roi_shapely)]
Censo_2017 = Censo_2017[Censo_2017.geometry.within(roi_shapely)]
Comercio = Comercio[Comercio.geometry.within(roi_shapely)]
Ejes = Ejes[Ejes.geometry.within(roi_shapely)]
EstablecimientosSalud = EstablecimientosSalud[EstablecimientosSalud.geometry.within(roi_shapely)]
Manzanas = Manzanas[Manzanas.geometry.within(roi_shapely)]
Paraderos = Paraderos[Paraderos.geometry.within(roi_shapely)]

### Comunas

Filtramos que el Limite comunal solo incluya nuestra zona de interes

In [12]:
Limite_Comunal['geometry'] = Limite_Comunal.geometry.intersection(roi_shapely)
Limite_Comunal = Limite_Comunal[~Limite_Comunal.is_empty]

In [13]:
m = geemap.Map(center=[-33.45, -70.65], zoom=12)
m.add_gMetro(stgo, "Santiago Urbano", style={'color': 'blue', 'fillOpacity': 0.1})
m.add_gdf(Limite_Comunal, "Comunas Recortadas", style={'color': 'black', 'fillOpacity': 0.4, 'weight': 2})
m

Map(center=[-33.45, -70.65], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

## Metro

In [ ]:
df = pd.read_csv("Datos/plantilla_metro.csv", decimal=',')
for col in ['Lat Origen', 'Long Origen', 'Lat Destino', 'Long Destino']:
    df[col] = df[col].astype(str).str.replace(',', '.').astype(float)

df['geometry'] = df.apply(
    lambda row: LineString([
        (row['Long Origen'], row['Lat Origen']),
        (row['Long Destino'], row['Lat Destino'])
    ]),
    axis=1
)

Metro = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")
Metro['linea'] = Metro['linea'].astype(str)

# Colores válidos (sin #)
colores_lineas = {
    '1': 'E1251B',
    '2': 'FFD600',
    '3': '682600',
    '4': '0072BC',
    '4A': '5FAEE2',
    '5': '365A05',
    '6': 'B303DF'
}
Metro['color'] = Metro['linea'].map(colores_lineas)

# Convertir a EE.FeatureCollection con estilos
features = []
for _, row in Metro.iterrows():
    geom = geemap.geopandas_to_ee(gpd.GeoDataFrame([row], crs="EPSG:4326")).geometry()
    feature = ee.Feature(geom).set('style', {
        'color': row['color'],
        'width': 4,
        'lineType': 'solid'
    })
    features.append(feature)

fc = ee.FeatureCollection(features)

# Agregar al mapa
m = geemap.Map(center=[-33.45, -70.65], zoom=12)
m.addLayer(fc.style(**{'styleProperty': 'style'}), {}, 'Lineas de Metro')
m


Map(center=[-33.45, -70.65], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [17]:
from shapely.geometry import Point

estaciones = pd.concat([
    Metro[['Estación Origen', 'Lat Origen', 'Long Origen']].rename(
        columns={'Estación Origen': 'Estacion', 'Lat Origen': 'Lat', 'Long Origen': 'Lon'}),
    Metro[['Estación Destino', 'Lat Destino', 'Long Destino']].rename(
        columns={'Estación Destino': 'Estacion', 'Lat Destino': 'Lat', 'Long Destino': 'Lon'})
], ignore_index=True).drop_duplicates(subset=['Estacion'])

estaciones['geometry'] = estaciones.apply(lambda row: Point(row['Lon'], row['Lat']), axis=1)
gdf_estaciones = gpd.GeoDataFrame(estaciones, geometry='geometry', crs="EPSG:4326")

gdf_estaciones_comunas = gpd.sjoin(gdf_estaciones, Limite_Comunal, how='left', predicate='within')

conteo = gdf_estaciones_comunas['comuna'].value_counts().sort_index()

comunas_todas = Limite_Comunal['comuna'].unique()

comunas_sin_metro = set(comunas_todas) - set(conteo.index)

#print("Estaciones de metro por comuna:")
#print(conteo)
print("Comunas sin estaciones de metro:")
print(sorted(comunas_sin_metro))


Comunas sin estaciones de metro:
['CERRO NAVIA', 'COLINA', 'HUECHURABA', 'LA PINTANA', 'LAMPA', 'LO BARNECHEA', 'LO ESPEJO', 'RENCA', 'VITACURA']


In [19]:
# Crear personas_por_comuna agrupando por nombre de comuna
personas_por_comuna = Manzanas.groupby('DES_COMU')['PERSONAS'].sum()

# Filtrar las comunas sin metro en personas_por_comuna
personas_sin_metro = personas_por_comuna[personas_por_comuna.index.isin(comunas_sin_metro)]
print(personas_sin_metro)
print(f"Total de personas en comunas sin metro: {personas_sin_metro.sum()}")

DES_COMU
CERRO NAVIA     120684
HUECHURABA       51721
LA PINTANA      174065
LAMPA                4
LO BARNECHEA     60448
LO ESPEJO        98651
RENCA           110913
VITACURA         76696
Name: PERSONAS, dtype: int64
Total de personas en comunas sin metro: 693182
